Modelo de prueba

In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
import pandas as pd
import sys

sys.path.append('../src') 
from app import get_api_data

api = get_api_data()

# 2. Extraer campos relevantes
rows = []
for country in api:
    name = country.get('name', {}).get('common', 'N/A')
    capital = country.get('capital', ['N/A'])[0]
    region = country.get('region', 'N/A')
    population = country.get('population', None)
    area = country.get('area', None)
    languages = ', '.join(country.get('languages', {}).values()) if 'languages' in country else None

    # Validamos que existan todos los campos numéricos y categóricos necesarios
    if region and population and area and languages:
        rows.append({
            'País': name,
            'Capital': capital,
            'Región': region,
            'Población': population,
            'Área (km²)': area,
            'Idiomas': languages
        })

# 3. Crear DataFrame
df = pd.DataFrame(rows)

# 4. Feature engineering
df['Densidad'] = df['Población'] / df['Área (km²)']  # nueva columna: densidad poblacional
df['Idioma_principal'] = df['Idiomas'].apply(lambda x: x.split(',')[0].strip())  # primer idioma

# 5. Filtrar regiones válidas
df = df[df['Región'].isin(['Africa', 'Asia', 'Europe', 'Oceania', 'Americas'])]

# 6. One-hot encoding del idioma principal
idiomas_dummies = pd.get_dummies(df['Idioma_principal'], prefix='lang')

# 7. Variables numéricas + codificadas
X = pd.concat([df[['Población', 'Área (km²)', 'Densidad']], idiomas_dummies], axis=1)
y = df['Región']

# 8. Escalado de variables numéricas
scaler = StandardScaler()
X[['Población', 'Área (km²)', 'Densidad']] = scaler.fit_transform(X[['Población', 'Área (km²)', 'Densidad']])

# 9. División en entrenamiento/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 10. Entrenar modelo
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)

# 11. Predicción y evaluación
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))



              precision    recall  f1-score   support

      Africa       0.60      0.40      0.48        15
    Americas       0.36      0.67      0.47         6
        Asia       0.67      0.62      0.64        13
      Europe       0.46      0.75      0.57         8
     Oceania       0.67      0.29      0.40         7

    accuracy                           0.53        49
   macro avg       0.55      0.54      0.51        49
weighted avg       0.58      0.53      0.52        49



Data Frame

In [14]:

api = get_api_data()


# 2. Extraer campos relevantes
rows = []
for country in api:
    name_common = country.get('name', {}).get('common', 'N/A')
    independent = country.get('independent', 'N/A')
    un_member = country.get('unMember', 'N/A')
    capital = country.get('capital', ['N/A'])[0]
    region = country.get('region', 'N/A')
    subregion = country.get('subregion', 'N/A')
    languages = ', '.join(country.get('languages', {}).values()) if 'languages' in country else 'N/A'
    latlng = country.get('latlng', 'N/A')
    landlocked = country.get('landlocked', 'N/A')
    area = country.get('area', 'N/A')
    population = country.get('population', 'N/A')
    gini_2017 = country.get('gini', {}).get('2017', 'N/A')
    car_side = country.get('car', {}).get('side', 'N/A') if 'car' in country else 'N/A'
    timezones = ', '.join(country.get('timezones', []))
    continents = ', '.join(country.get('continents', []))
    start_of_week = country.get('startOfWeek', 'N/A')
    
    rows.append({
        'name.common': name_common,
        'Independent': independent,
        'unMember': un_member,
        'capital': capital,
        'region': region,
        'subregion': subregion,
        'languaje': languages,
        'latlng': latlng,
        'landlocked': landlocked,
        'area': area,
        'population': population,
        'gini.2017': gini_2017,
        'car.side': car_side,
        'timezones': timezones,
        'continents': continents,
        'startOfWeek': start_of_week
    })

# 3. Crear el DataFrame
df = pd.DataFrame(rows)

# 4. Mostrar el DataFrame de manera más ordenada y visual
pd.set_option('display.max_columns', None)  # Asegura que todas las columnas se muestren
pd.set_option('display.width', 1000)        # Aumenta el ancho de la visualización
pd.set_option('display.max_rows', 253)      # Limita la cantidad de filas mostradas

# 5. Mostrar las primeras filas del DataFrame
print(df.head(253))  # Puedes ajustar el número de filas que quieres ver



JSONDecodeError: Expecting value: line 1 column 1 (char 0)

Crear CSV

In [15]:
import csv

api = get_api_data()

# Extraer los datos deseados
records = []
for country in api:
    record = {
        "name.common": country.get("name", {}).get("common", None),
        "independent": country.get("independent", None),
        "unMember": country.get("unMember", None),
        "capital": ", ".join(country.get("capital", [])) if country.get("capital") else None,
        "region": country.get("region", None),
        "subregion": country.get("subregion", None),
        "language": ", ".join(country.get("languages", {}).values()) if country.get("languages") else None,
        "latlng": str(country.get("latlng", None)),
        "landlocked": country.get("landlocked", None),
        "area": country.get("area", None),
        "population": country.get("population", None),
        "borders": ", ".join(country.get("borders", [])) if country.get("borders") else None,
        "car.side": country.get("car", {}).get("side", None),
        "timezones": ", ".join(country.get("timezones", [])) if country.get("timezones") else None,
        "continents": ", ".join(country.get("continents", [])) if country.get("continents") else None,
        "startOfWeek": country.get("startOfWeek", None)
    }
    records.append(record)

# Guardar en archivo CSV
with open("paises_con_borders.csv", mode="w", newline="", encoding="utf-8") as file:
    writer = csv.DictWriter(file, fieldnames=records[0].keys())
    writer.writeheader()
    writer.writerows(records)

print("✅ Archivo CSV generado: paises_con_borders.csv")


✅ Archivo CSV generado: paises_con_borders.csv


Modelo (aun no terminado)

In [22]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score

# 1. Cargar el dataset
df = pd.read_csv('paises_todos_atributos.csv')

# 2. Eliminar columnas no útiles o duplicadas si existen
df = df.drop(columns=[col for col in df.columns if 'unnamed' in col.lower()], errors='ignore')

# 3. Codificar variables categóricas
label_cols = ['name.common', 'independent', 'unMember', 'capital', 'region', 'subregion', 'language', 'latlng', 'landlocked', 'area', 'population', 'gini.2017', 'car.side', 'timezones', 'continents', 'startOfWeek']
le_dict = {}

for col in label_cols:
    le = LabelEncoder()
    df[col] = df[col].fillna('Desconocido').astype(str)
    df[col] = le.fit_transform(df[col])
    le_dict[col] = le  # Guardamos los encoders por si queremos predecir luego

# 4. Rellenar valores nulos restantes
df['gini.2017'] = df['gini.2017'].fillna(df['gini.2017'].median())

# 5. Definir variables predictoras (X) y variable objetivo (y)
X = df.drop(columns=['unMember'])  # Variable objetivo
y = df['unMember']

# 6. Separar en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 7. Entrenar el modelo
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 8. Evaluar
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

print(df.columns.tolist())



Accuracy: 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      1.00      1.00        40

    accuracy                           1.00        50
   macro avg       1.00      1.00      1.00        50
weighted avg       1.00      1.00      1.00        50

['name.common', 'independent', 'unMember', 'capital', 'region', 'subregion', 'language', 'latlng', 'landlocked', 'area', 'population', 'gini.2017', 'car.side', 'timezones', 'continents', 'startOfWeek']


ENCODER

In [2]:
from sklearn.preprocessing import LabelEncoder
import joblib
import pandas as pd

# 1. Cargar tu conjunto de datos
df = pd.read_csv('paises_con_borders.csv')

# 2. Crear un diccionario de LabelEncoders para cada columna categórica
columns_to_encode = ['region', 'subregion', 'language', 'car.side', 'timezones', 'continents', 'startOfWeek', 'latlng']  # Ajustamos para incluir 'latlng'
le_dict = {}

# 3. Ajustar LabelEncoder para cada columna categórica
for col in columns_to_encode:
    le = LabelEncoder()
    df[col] = df[col].fillna('Desconocido')  # Reemplazar valores NaN por 'Desconocido'
    
    # Si la columna es 'latlng' y tiene coordenadas (como una lista de latitud y longitud)
    if col == 'latlng':
        # Para latlng, si es una cadena de coordenadas separadas por coma (ejemplo: '12.34,-56.78'), podemos convertirlo en una cadena única
        df[col] = df[col].apply(lambda x: x if isinstance(x, str) else 'Desconocido')  # Si no es una cadena, lo marcamos como 'Desconocido'
    
    # Ajustar y transformar los datos
    df[col] = le.fit_transform(df[col])  # Ajustar y transformar los datos
    le_dict[col] = le  # Guardar el encoder para futuras predicciones

# 4. Guardar los encoders para usarlos en el futuro
joblib.dump(le_dict, 'encoders.pickle')

# 5. Verifica que el DataFrame ahora tiene las columnas transformadas
print(df.head())


        name.common  independent  unMember     capital  region  subregion  \
0          Botswana         True      True    Gaborone       0         19   
1             Tonga         True      True  Nuku'alofa       5         15   
2            Greece         True      True      Athens       4         21   
3  Marshall Islands         True      True      Majuro       5         10   
4           Belarus         True      True       Minsk       4          8   

   language  latlng  landlocked      area  population  \
0        75      32        True  582000.0     2351625   
1        74      24       False     747.0      105697   
2        93     164       False  131990.0    10715549   
3        62     248       False     181.0       59194   
4        18     216        True  207600.0     9398861   

                   borders  car.side  timezones  continents  startOfWeek  
0       NAM, ZAF, ZMB, ZWE         0          4           0            0  
1                      NaN         0        

In [4]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
import joblib

# 1. Cargar el modelo entrenado y los encoders
model = joblib.load("modelo_un.pickle")
le_dict = joblib.load("encoders.pickle")

# 2. Cargar el CSV con los datos de los países
df = pd.read_csv('paises_con_borders.csv')  # Reemplaza con el nombre de tu archivo CSV

# 3. Columnas requeridas
cols = ['name.common', 'independent', 'capital', 'region', 'subregion', 'language', 'latlng', 'landlocked',
        'area', 'population', 'gini.2017', 'car.side', 'timezones', 'continents', 'startOfWeek']

# 4. Especifica aquí el nombre del país para hacer la predicción
pais = "Chile"  # Cambia "Chile" por el nombre del país que deseas predecir

# 5. Función para predecir si un país debería ser miembro de la ONU
def predecir_pais():
    # Filtrar el dataframe para obtener los datos del país
    input_df = df[df['name.common'] == pais]

    if input_df.empty:
        print(f"\n🔴 No se encontró el país '{pais}' en los datos.")
        return

    # Solo tomar las columnas necesarias
    input_df = input_df[cols]

    # Convertir tipos según necesidad
    input_df['area'] = input_df['area'].astype(float)
    input_df['population'] = input_df['population'].astype(int)
    input_df['gini.2017'] = input_df['gini.2017'].fillna(0.0).astype(float)

    # Aplicar los mismos encoders
    for col in le_dict:
        if col in input_df.columns:
            le = le_dict[col]
            input_df[col] = input_df[col].fillna('Desconocido').astype(str)
            input_df[col] = le.transform(input_df[col])

    # Asegurar que tenga las columnas correctas
    input_df = input_df[model.feature_names_in_]

    # Predecir
    pred = model.predict(input_df)[0]
    print("\n➡ El modelo predice que este país " +
          ("🟢 DEBERÍA" if pred == 1 else "🔴 NO DEBERÍA") + " ser miembro de las Naciones Unidas.")

# 6. Ejecutar la función
predecir_pais()


KeyError: "['gini.2017'] not in index"

In [6]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
import joblib
from sklearn.metrics import classification_report, accuracy_score

# 1. Cargar el modelo entrenado y los encoders (si los tienes guardados)
# Si aún no tienes el archivo de encoders, no lo cargues, creémoslo al vuelo
try:
    model = joblib.load("modelo_un.pickle")
    le_dict = joblib.load("encoders.pickle")
except FileNotFoundError:
    print("🔴 No se encontraron los archivos del modelo o los encoders guardados.")
    le_dict = {}

# 2. Cargar el CSV con los datos de los países
df = pd.read_csv('paises_con_borders.csv')  # Reemplaza con el nombre de tu archivo CSV

# 3. Columnas requeridas
cols = ['name.common', 'independent','unMember', 'capital', 'region', 'subregion', 'language', 'latlng', 'landlocked',
        'area', 'population', 'gini.2017', 'car.side', 'timezones', 'continents', 'startOfWeek']

# 4. Especifica aquí el nombre del país para hacer la predicción
pais = "Chile"  # Cambia "Chile" por el nombre del país que deseas predecir

# 5. Cargar el conjunto de datos de prueba (si tienes uno)
df_test = pd.read_csv('paises_con_borders.csv')  # Reemplaza con tu archivo de prueba

# 6. Verificar que las columnas necesarias están presentes en df_test
if not all(col in df_test.columns for col in cols):
    print("🔴 Algunas columnas necesarias no están presentes en el conjunto de datos de prueba.")
else:
    # 7. Filtrar solo las columnas necesarias del conjunto de datos de prueba
    df_test = df_test[cols]

    # Crear LabelEncoders si no existen
    columns_to_encode = ['region', 'subregion', 'language', 'car.side', 'timezones', 'continents', 'startOfWeek']
    if not le_dict:  # Si no tenemos encoders cargados, los creamos
        le_dict = {}
        for col in columns_to_encode:
            le = LabelEncoder()
            df[col] = df[col].fillna('Desconocido')  # Reemplazar valores NaN por 'Desconocido'
            df[col] = le.fit_transform(df[col])  # Ajustar y transformar los datos
            le_dict[col] = le  # Guardar el encoder para futuras predicciones
        # Guardar los encoders
        joblib.dump(le_dict, 'encoders.pickle')

    # 8. Función para predecir si un país debería ser miembro de la ONU
    def predecir_pais():
        # Filtrar el dataframe para obtener los datos del país
        input_df = df[df['name.common'] == pais]

        if input_df.empty:
            print(f"\n🔴 No se encontró el país '{pais}' en los datos.")
            return

        # Solo tomar las columnas necesarias
        input_df = input_df[cols]

        # Convertir tipos según necesidad
        input_df['area'] = input_df['area'].astype(float)
        input_df['population'] = input_df['population'].astype(int)
        input_df['gini.2017'] = input_df['gini.2017'].fillna(0.0).astype(float)

        # Aplicar los mismos encoders
        for col in le_dict:
            if col in input_df.columns:
                le = le_dict[col]
                input_df[col] = input_df[col].fillna('Desconocido').astype(str)
                input_df[col] = le.transform(input_df[col])

        # Asegurar que tenga las columnas correctas
        input_df = input_df[model.feature_names_in_]

        # Predecir
        pred = model.predict(input_df)[0]
        print("\n➡ El modelo predice que este país " +
              ("🟢 DEBERÍA" if pred == 1 else "🔴 NO DEBERÍA") + " ser miembro de las Naciones Unidas.")

        # Asegurarse de que las columnas categóricas de df_test estén correctamente transformadas
        for col in le_dict:
            if col in df_test.columns:
                le = le_dict[col]
                df_test[col] = df_test[col].fillna('Desconocido').astype(str)
                df_test[col] = le.transform(df_test[col])

        # Convertir las columnas numéricas de texto a números
        df_test['area'] = df_test['area'].astype(float)
        df_test['population'] = df_test['population'].astype(int)
        df_test['gini.2017'] = df_test['gini.2017'].fillna(0.0).astype(float)

        # Asegurarse de que las columnas estén alineadas con las características de entrada del modelo
        df_test = df_test[model.feature_names_in_]

        # Verificar que la columna 'unMember' esté en el conjunto de prueba
        if 'unMember' not in df_test.columns:
            print("🔴 La columna 'unMember' no está presente en el conjunto de prueba.")
            return

        # Calcular y mostrar las métricas de clasificación en el conjunto de datos de prueba
        y_true = df_test['unMember']  # Asegúrate de que la columna 'unMember' esté en tus datos de prueba
        y_pred = model.predict(df_test)

        print("\nMétricas de rendimiento:")
        print(f"Accuracy: {accuracy_score(y_true, y_pred):.2f}")
        print(classification_report(y_true, y_pred))

    # 9. Ejecutar la función
    predecir_pais()


🔴 Algunas columnas necesarias no están presentes en el conjunto de datos de prueba.


In [8]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
import joblib
from sklearn.metrics import classification_report, accuracy_score

# 1. Cargar modelo y encoders
def cargar_modelo_y_encoders():
    try:
        model = joblib.load("modelo_un.pickle")
        le_dict = joblib.load("encoders.pickle")
    except FileNotFoundError:
        print("🔴 No se encontraron los archivos del modelo o los encoders guardados.")
        model = None
        le_dict = {}
    return model, le_dict

# 2. Preparar el dataset
def preparar_datos(filepath, cols, le_dict):
    df = pd.read_csv(filepath)

    if not all(col in df.columns for col in cols):
        print("🔴 Algunas columnas necesarias no están presentes en el conjunto de datos.")
        return None, None

    df = df[cols].copy()
    
    # Codificar variables categóricas
    columns_to_encode = ['region', 'subregion', 'language', 'car.side', 'timezones', 'continents', 'startOfWeek', 'latlng']
    if not le_dict:
        le_dict = {}
        for col in columns_to_encode:
            le = LabelEncoder()
            df[col] = df[col].fillna('Desconocido').astype(str)
            df[col] = le.fit_transform(df[col])
            le_dict[col] = le
        joblib.dump(le_dict, 'encoders.pickle')
    else:
        for col in le_dict:
            if col in df.columns:
                le = le_dict[col]
                df[col] = df[col].fillna('Desconocido').astype(str)
                df[col] = le.transform(df[col])

    # Asegurar tipo numérico
    df['area'] = df['area'].astype(float)
    df['population'] = df['population'].astype(int)

    return df, le_dict

# 3. Función de predicción
def predecir_pais(pais, model, df_full, le_dict):
    input_df = df_full[df_full['name.common'] == pais].copy()

    if input_df.empty:
        print(f"\n🔴 No se encontró el país '{pais}' en los datos.")
        return

    try:
        input_df = input_df[model.feature_names_in_]
        pred = model.predict(input_df)[0]
        print("\n➡ El modelo predice que este país " +
              ("🟢 DEBERÍA" if pred == 1 else "🔴 NO DEBERÍA") +
              " ser miembro de las Naciones Unidas.")
    except Exception as e:
        print("🔴 Error durante la predicción:", e)

# 4. Evaluación del modelo completo
def evaluar_modelo(model, df_test):
    if 'unMember' not in df_test.columns:
        print("🔴 La columna 'unMember' no está presente en el conjunto de prueba.")
        return

    try:
        y_true = df_test['unMember'].copy()  # ← Guardar antes de eliminar columnas
        df_test = df_test[model.feature_names_in_]
        y_pred = model.predict(df_test)

        print("\n📊 Métricas del modelo:")
        print(f"Accuracy: {accuracy_score(y_true, y_pred):.2f}")
        print(classification_report(y_true, y_pred))
    except Exception as e:
        print("🔴 Error al evaluar el modelo:", e)


# =============================
# Ejecutar todo el pipeline
# =============================
if __name__ == "__main__":
    cols = ['name.common', 'independent', 'unMember', 'capital', 'region', 'subregion', 'language', 'latlng',
            'landlocked', 'area', 'population', 'car.side', 'timezones', 'continents', 'startOfWeek']
    pais = "Hong Kong"  # Cambia el país aquí si lo deseas
    filepath = 'paises_con_borders.csv'

    model, le_dict = cargar_modelo_y_encoders()

    if model:
        df_full, le_dict = preparar_datos(filepath, cols, le_dict)
        if df_full is not None:
            predecir_pais(pais, model, df_full, le_dict)
            evaluar_modelo(model, df_full)



➡ El modelo predice que este país 🔴 NO DEBERÍA ser miembro de las Naciones Unidas.

📊 Métricas del modelo:
Accuracy: 1.00
              precision    recall  f1-score   support

       False       1.00      1.00      1.00        58
        True       1.00      1.00      1.00       192

    accuracy                           1.00       250
   macro avg       1.00      1.00      1.00       250
weighted avg       1.00      1.00      1.00       250



In [9]:
from sklearn.preprocessing import LabelEncoder
import joblib
import pandas as pd

# 1. Cargar tu conjunto de datos
df = pd.read_csv('paises_con_borders.csv')

# 2. Crear un diccionario de LabelEncoders para cada columna categórica
columns_to_encode = ['region', 'subregion', 'language', 'car.side', 'timezones', 'continents', 'startOfWeek', 'latlng']
le_dict = {}

# 3. Ajustar LabelEncoder para cada columna categórica
for col in columns_to_encode:
    le = LabelEncoder()
    df[col] = df[col].fillna('Desconocido').astype(str)  # Reemplazar NaN y asegurar tipo string
    df[col] = le.fit_transform(df[col])  # Ajustar y transformar los datos
    le_dict[col] = le  # Guardar el encoder para futuras predicciones

# 4. Guardar los encoders para usarlos en el futuro
joblib.dump(le_dict, 'encoders.pickle')

# 5. Verifica que el DataFrame ahora tiene las columnas transformadas
print(df.head())


        name.common  independent  unMember     capital  region  subregion  \
0          Botswana         True      True    Gaborone       0         19   
1             Tonga         True      True  Nuku'alofa       5         15   
2            Greece         True      True      Athens       4         21   
3  Marshall Islands         True      True      Majuro       5         10   
4           Belarus         True      True       Minsk       4          8   

   language  latlng  landlocked      area  population  \
0        75      32        True  582000.0     2351625   
1        74      24       False     747.0      105697   
2        93     164       False  131990.0    10715549   
3        62     248       False     181.0       59194   
4        18     216        True  207600.0     9398861   

                   borders  car.side  timezones  continents  startOfWeek  
0       NAM, ZAF, ZMB, ZWE         0          4           0            0  
1                      NaN         0        

In [11]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
import joblib
from sklearn.metrics import classification_report, accuracy_score

# 1. Cargar modelo y encoders
def cargar_modelo_y_encoders():
    try:
        model = joblib.load("modelo_un.pickle")
        le_dict = joblib.load("encoders.pickle")
    except FileNotFoundError:
        print("🔴 No se encontraron los archivos del modelo o los encoders guardados.")
        model = None
        le_dict = {}
    return model, le_dict

# 2. Preparar el dataset
def preparar_datos(filepath, cols, le_dict):
    df = pd.read_csv(filepath)

    if not all(col in df.columns for col in cols):
        print("🔴 Algunas columnas necesarias no están presentes en el conjunto de datos.")
        return None, None, None

    df = df[cols].copy()

    # Guardar target y quitarlo de las features
    y_true = df['unMember'].copy()
    df = df.drop(columns=['unMember'])

    # Codificar variables categóricas
    columns_to_encode = ['region', 'subregion', 'language', 'car.side', 'timezones', 'continents', 'startOfWeek', 'latlng']
    if not le_dict:
        le_dict = {}
        for col in columns_to_encode:
            le = LabelEncoder()
            df[col] = df[col].fillna('Desconocido').astype(str)
            df[col] = le.fit_transform(df[col])
            le_dict[col] = le
        joblib.dump(le_dict, 'encoders.pickle')
    else:
        for col in le_dict:
            if col in df.columns:
                le = le_dict[col]
                df[col] = df[col].fillna('Desconocido').astype(str)
                df[col] = le.transform(df[col])

    # Asegurar tipo numérico
    df['area'] = df['area'].astype(float)
    df['population'] = df['population'].astype(int)

    return df, y_true, le_dict

# 3. Función de predicción
def predecir_pais(pais, model, df_full_original, le_dict):
    input_df = df_full_original[df_full_original['name.common'] == pais].copy()

    if input_df.empty:
        print(f"\n🔴 No se encontró el país '{pais}' en los datos.")
        return

    try:
        input_df = input_df[model.feature_names_in_]
        pred = model.predict(input_df)[0]
        print("\n➡ El modelo predice que este país " +
              ("🟢 DEBERÍA" if pred == 1 else "🔴 NO DEBERÍA") +
              " ser miembro de las Naciones Unidas.")
    except Exception as e:
        print("🔴 Error durante la predicción:", e)

# 4. Evaluación del modelo completo
def evaluar_modelo(model, df_test, y_true):
    try:
        df_test = df_test[model.feature_names_in_]
        y_pred = model.predict(df_test)

        print("\n📊 Métricas del modelo:")
        print(f"Accuracy: {accuracy_score(y_true, y_pred):.2f}")
        print(classification_report(y_true, y_pred))
    except Exception as e:
        print("🔴 Error al evaluar el modelo:", e)


# =============================
# Ejecutar todo el pipeline
# =============================
if __name__ == "__main__":
    cols = ['name.common', 'independent', 'unMember', 'capital', 'region', 'subregion', 'language', 'latlng',
            'landlocked', 'area', 'population', 'car.side', 'timezones', 'continents', 'startOfWeek']
    pais = "Sint Maarten"  # Cambia el país aquí si lo deseas
    filepath = 'paises_con_borders.csv'

    model, le_dict = cargar_modelo_y_encoders()

    if model:
        df_features, y_true, le_dict = preparar_datos(filepath, cols, le_dict)
        if df_features is not None:
            predecir_pais(pais, model, df_features, le_dict)
            evaluar_modelo(model, df_features, y_true)



➡ El modelo predice que este país 🔴 NO DEBERÍA ser miembro de las Naciones Unidas.

📊 Métricas del modelo:
Accuracy: 1.00
              precision    recall  f1-score   support

       False       1.00      1.00      1.00        58
        True       1.00      1.00      1.00       192

    accuracy                           1.00       250
   macro avg       1.00      1.00      1.00       250
weighted avg       1.00      1.00      1.00       250



In [19]:


# Extraer los datos deseados
records = []
for country in api:
    latlng = country.get("latlng", [None, None])
    lat = latlng[0] if len(latlng) > 0 else None
    lng = latlng[1] if len(latlng) > 1 else None

    record = {
        "name.common": country.get("name", {}).get("common", None),
        "independent": country.get("independent", None),
        "unMember": country.get("unMember", None),
        "capital": ", ".join(country.get("capital", [])) if country.get("capital") else None,
        "region": country.get("region", None),
        "subregion": country.get("subregion", None),
        "language": ", ".join(country.get("languages", {}).values()) if country.get("languages") else None,
        "latitude": lat,
        "longitude": lng,
        "landlocked": country.get("landlocked", None),
        "area": country.get("area", None),
        "population": country.get("population", None),
        "borders": ", ".join(country.get("borders", [])) if country.get("borders") else None,
        "car.side": country.get("car", {}).get("side", None),
        "timezones": ", ".join(country.get("timezones", [])) if country.get("timezones") else None,
        "continents": ", ".join(country.get("continents", [])) if country.get("continents") else None,
        "startOfWeek": country.get("startOfWeek", None)
    }
    records.append(record)

# Guardar en archivo CSV
with open("paises_con_lat_lon.csv", mode="w", newline="", encoding="utf-8") as file:
    writer = csv.DictWriter(file, fieldnames=records[0].keys())
    writer.writeheader()
    writer.writerows(records)

print("✅ Archivo CSV generado: paises_con_lat_lon.csv")


✅ Archivo CSV generado: paises_con_lat_lon.csv


In [21]:

# Diccionarios para codificar variables categóricas
region_codes = {}
subregion_codes = {}
language_codes = {}
capital_codes = {}
continent_codes = {}
start_of_week_codes = {}
car_side_codes = {}

def get_code(value, code_dict):
    if value not in code_dict:
        code_dict[value] = len(code_dict) + 1
    return code_dict[value]

# Extraer y codificar datos
records = []
for country in api:
    latlng = country.get("latlng", [None, None])
    lat = latlng[0] if len(latlng) > 0 else None
    lng = latlng[1] if len(latlng) > 1 else None

    record = {
        "independent": int(country.get("independent", False)),
        "unMember": int(country.get("unMember", False)),
        "capital_code": get_code(", ".join(country.get("capital", [])) if country.get("capital") else "None", capital_codes),
        "region_code": get_code(country.get("region", "None"), region_codes),
        "subregion_code": get_code(country.get("subregion", "None"), subregion_codes),
        "language_code": get_code(", ".join(country.get("languages", {}).values()) if country.get("languages") else "None", language_codes),
        "latitude": lat,
        "longitude": lng,
        "landlocked": int(country.get("landlocked", False)),
        "area": country.get("area", 0),
        "population": country.get("population", 0),
        "borders_count": len(country.get("borders", [])),
        "car_side_code": get_code(country.get("car", {}).get("side", "None"), car_side_codes),
        "timezones_count": len(country.get("timezones", [])),
        "continents_code": get_code(", ".join(country.get("continents", [])) if country.get("continents") else "None", continent_codes),
        "startOfWeek_code": get_code(country.get("startOfWeek", "None"), start_of_week_codes)
    }
    records.append(record)

# Guardar en archivo CSV
with open("paises_numericos.csv", mode="w", newline="", encoding="utf-8") as file:
    writer = csv.DictWriter(file, fieldnames=records[0].keys())
    writer.writeheader()
    writer.writerows(records)

print("✅ Archivo CSV generado: paises_numericos.csv")


✅ Archivo CSV generado: paises_numericos.csv
